# MNIST Large Untrained Net Exc Inh

Derived from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb

In [263]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from keras import backend as K
#print("keras version:",tf.keras.__version__)

TensorFlow version: 2.8.2


In [264]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [265]:
positiveWeightImplementation = False	#orig: True
if(not positiveWeightImplementation):
    integrateWeights = True
    if(integrateWeights):
        integrateWeights1 = False    #explicitly declare E/I neurons
        integrateWeights2 = True    #implicitly declare E/I neurons        

generateUntrainedNetwork = False
if(generateUntrainedNetwork):
    #only train the last layer
    numberOfHiddenLayers = 2    #default: 2    #if 0 then useSVM=True
else:
    numberOfHiddenLayers = 2    #default: 2

if(numberOfHiddenLayers > 1):
    addSkipLayers = False   #optional
else:
    addSkipLayers = False   #mandatory

layerSizeBase = 128  #default: 128

batch_size = 64 #default: 64
epochs = 5  #1  #5

debugNoEIneurons = False
debugPreTrainWeights = True
debugPreTrainOutputs = True
debugPostTrainWeights = True
debugPostTrainOutputs = True

## Load data

In [266]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train.shape = ", x_train.shape)

input_shape = (28, 28)

x_train.shape =  (60000, 28, 28)


## Define model

In [267]:
def activation(x):
    return K.maximum(x, 0)  #ReLU
    
def activationExcitatory(x):
    return K.maximum(x, 0)  #ReLU

def activationInhibitory(x):
    if(positiveWeightImplementation):
        return -(K.maximum(x, 0))   #ReLU with negative output
    else:
        return K.maximum(x, 0)  #ReLU

def neuronInitializer(shape, dtype=None):
    if(positiveWeightImplementation):
        print("neuronInitializer error: requires !positiveWeightImplementation:integrateWeights")
    else:
        if(integrateWeights):
            w = tf.math.abs(tf.random.normal(shape, dtype=dtype))
            wEIsize = w.shape[0]//2
            wSignE = tf.ones([wEIsize, w.shape[1]])
            wSignI = tf.ones([wEIsize, w.shape[1]])
            wSignI = tf.multiply(wSignI, -1)
            wSign = tf.concat([wSignE, wSignI], axis=0)
            w = tf.multiply(w, wSign)
        else:
            print("neuronInitializer error: requires !positiveWeightImplementation:integrateWeights")
    return w

def excitatoryNeuronInitializer(shape, dtype=None):
    return tf.math.abs(tf.random.normal(shape, dtype=dtype))

def inhibitoryNeuronInitializer(shape, dtype=None):
    if(positiveWeightImplementation):
        return tf.math.abs(tf.random.normal(shape, dtype=dtype))
    else:
        return tf.math.negative(tf.math.abs(tf.random.normal(shape, dtype=dtype)))

class negative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        return w * tf.cast(tf.math.less_equal(w, 0.), w.dtype)

class positiveOrNegative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        w_shape = w.shape
        wEIsize = w.shape[0]//2
        wE = w[0:wEIsize]
        wI = w[wEIsize:]
        wEcheck = tf.greater_equal(wE, 0)
        wIcheck = tf.less_equal(wI, 0)
        wEcheck = tf.cast(wEcheck, tf.float32)
        wIcheck = tf.cast(wIcheck, tf.float32)
        wE = tf.multiply(wE, wEcheck)
        wI = tf.multiply(wI, wIcheck)
        w = tf.concat([wE, wI], axis=0)
        return w


In [268]:
num_classes = 10

if(positiveWeightImplementation):
    weightConstraint = tf.keras.constraints.non_neg()
    constrainBiases = True   #ensure positive biases also
    if(constrainBiases):
        biasConstraint = tf.keras.constraints.non_neg()
        constrainBiasesLastLayer = False
        if(constrainBiasesLastLayer):
            biasConstraintLastLayer = tf.keras.constraints.non_neg()
        else:
            biasConstraintLastLayer = None
    else:
        biasConstraint = None
        biasConstraintLastLayer = None
    weightConstraintLastLayer = weightConstraint
else:
    if(integrateWeights):
        weightConstraint = positiveOrNegative()
        biasConstraint = None
        weightConstraintLastLayer = None
        biasConstraintLastLayer = None
    else:
        weightConstraintPositive = tf.keras.constraints.non_neg()
        weightConstraintNegative = negative()
        constrainBiases = False
        if(constrainBiases):
            biasConstraintPositive = tf.keras.constraints.non_neg()
            biasConstraintNegative = negative()
        else:
            biasConstraintPositive = None
            biasConstraintNegative = None
        weightConstraintLastLayer = None
        biasConstraintLastLayer = None

if(generateUntrainedNetwork):
    #only train the last layer
    generateLargeNetwork = True
else:
    generateLargeNetwork = False


if(generateLargeNetwork):
    generateLargeNetworkRatio = 50
    layerRatio = generateLargeNetworkRatio
else:
    layerRatio = 1  #10 #1

def createEIlayer(layerIndex, h0, firstLayer=False):
    if(debugNoEIneurons):
        h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #excitatory neuron inputs
        h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #inhibitory neuron inputs   
        h1E = tf.keras.layers.Activation(activationExcitatory)(h1E)
        h1I = tf.keras.layers.Activation(activationInhibitory)(h1I)
        h1 = tf.keras.layers.Concatenate()([h1E, h1I])  
    else:
        if(positiveWeightImplementation):
            h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)
            h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)
            h1E = tf.keras.layers.Activation(activationExcitatory)(h1E)
            h1I = tf.keras.layers.Activation(activationInhibitory)(h1I)
            h1 = tf.keras.layers.Concatenate()([h1E, h1I])
        else:
            if(integrateWeights):
                if(integrateWeights1):
                    if(firstLayer):
                        h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #excitatory neuron inputs
                        h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio)(h0)   #inhibitory neuron inputs             
                    else:
                        h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=neuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)   #excitatory neuron inputs
                        h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=neuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)   #inhibitory neuron inputs  
                    h1E = tf.keras.layers.Activation(activation)(h1E)
                    h1I = tf.keras.layers.Activation(activation)(h1I)
                    h1 = tf.keras.layers.Concatenate()([h1E, h1I])
                elif(integrateWeights2):
                    if(firstLayer):
                        h1 = tf.keras.layers.Dense(layerSizeBase*layerRatio*4)(h0)   #excitatory neuron inputs
                    else:
                        h1 = tf.keras.layers.Dense(layerSizeBase*layerRatio*2, kernel_initializer=neuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)   #excitatory neuron inputs
                        #h1 = tf.keras.layers.Dense(layerSizeBase*layerRatio*2, kernel_initializer=neuronInitializer)(h0)   #excitatory neuron inputs
                    h1 = tf.keras.layers.Activation(activation)(h1)
            else:
                if(firstLayer):
                    h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer)(h0)   #excitatory neuron inputs
                    h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer)(h0)   #inhibitory neuron inputs
                else:
                    h0E, h0I = h0
                    h1Ee = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraintPositive, bias_constraint=biasConstraintPositive)(h0E) #excitatory neuron excitatory inputs
                    h1Ei = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraintNegative, bias_constraint=biasConstraintNegative)(h0I) #excitatory neuron inhibitory inputs
                    h1Ie = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraintPositive, bias_constraint=biasConstraintPositive)(h0E) #inhibitory neuron excitatory inputs
                    h1Ii = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraintNegative, bias_constraint=biasConstraintNegative)(h0I) #inhibitory neuron inhibitory inputs
                    h1E = tf.keras.layers.Add()([h1Ee, h1Ei])
                    h1I = tf.keras.layers.Add()([h1Ie, h1Ii])
                h1E = tf.keras.layers.Activation(activationExcitatory)(h1E)
                h1I = tf.keras.layers.Activation(activationInhibitory)(h1I)
                h1 = (h1E, h1I)
    return h1

def concatEIneurons(h):
    if(positiveWeightImplementation):
        return h
    else: 
        if(integrateWeights):
            pass
        else:
            hE, hI = h
            h = tf.keras.layers.Concatenate()([hE, hI])
        return h

x = tf.keras.layers.Input(shape=input_shape)
h0 = tf.keras.layers.Flatten()(x)

hLast = h0
if(numberOfHiddenLayers >= 1):
    h1 = createEIlayer(1, h0, firstLayer=True)
    hLast = h1
if(numberOfHiddenLayers >= 2):
    h2 = createEIlayer(2, h1)
    hLast = h2
if(numberOfHiddenLayers >= 3):
    h3 = createEIlayer(3, h2)
    hLast = h3
if(numberOfHiddenLayers >= 4):
    h4 = createEIlayer(4, h3)
    hLast = h4
if(addSkipLayers):
    mList = []
    if(numberOfHiddenLayers >= 1):
        m1 = tf.keras.layers.Flatten()(concatEIneurons(h1))
        mList.append(m1)
    if(numberOfHiddenLayers >= 2):
        m2 = tf.keras.layers.Flatten()(concatEIneurons(h2))
        mList.append(m2)
    if(numberOfHiddenLayers >= 3):
        m3 = tf.keras.layers.Flatten()(concatEIneurons(h3))
        mList.append(m3)
    if(numberOfHiddenLayers >= 4):
        m4 = tf.keras.layers.Flatten()(concatEIneurons(h4))
        mList.append(m4)
    hLast = tf.keras.layers.concatenate(mList)
else:
    hLast = concatEIneurons(hLast)

if(generateUntrainedNetwork):
    hLast = tf.keras.layers.Lambda(lambda x: tf.keras.backend.stop_gradient(x))(hLast)
y = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_constraint=weightConstraintLastLayer, bias_constraint=biasConstraintLastLayer)(hLast)
model = tf.keras.Model(x, y)

#print(model.summary())
#model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
#evaluation accuracy: ? (with 1 or 2 hidden layers)


In [269]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [270]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

print(model.summary())

Model: "model_159"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 28, 28)]          0         
                                                                 
 flatten_23 (Flatten)        (None, 784)               0         
                                                                 
 dense_69 (Dense)            (None, 512)               401920    
                                                                 
 activation_46 (Activation)  (None, 512)               0         
                                                                 
 dense_70 (Dense)            (None, 256)               131328    
                                                                 
 activation_47 (Activation)  (None, 256)               0         
                                                                 
 dense_71 (Dense)            (None, 10)                25

In [271]:
if(debugPreTrainWeights):
    testwritefile = open('weightsPreTrain.txt', 'w')
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerWeights = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        #print(heading)
        weights = layer.get_weights()
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)
    testwritefile.close()

In [272]:
if(debugPreTrainOutputs):
    testwritefile = open('outputPreTrain.txt', 'w')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerOutputs = " + str(layerIndex) + "\n"
        #print(heading)
        testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        #print(layerOutput)
        layerOutputS =  str(layerOutput)
        testwritefile.write(layerOutputS)
    testwritefile.close()

## Train model

In [273]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/5
938/938 [==============================] - 3s 3ms/step - loss: 2.3038 - accuracy: 0.1118
Epoch 2/5
938/938 [==============================] - 2s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 3/5
938/938 [==============================] - 2s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 4/5
938/938 [==============================] - 2s 3ms/step - loss: 2.3013 - accuracy: 0.1124
Epoch 5/5
938/938 [==============================] - 2s 3ms/step - loss: 2.3013 - accuracy: 0.1124


In [274]:
if(debugPostTrainWeights):
    testwritefile = open('weightsPostTrain.txt', 'w')
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerWeights = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        #print(heading)
        weights = layer.get_weights()
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)
    testwritefile.close()

In [275]:
if(debugPostTrainOutputs):
    testwritefile = open('outputPostTrain.txt', 'w')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerOutputs = " + str(layerIndex) + "\n"
        #print(heading)
        testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        #print(layerOutput)
        layerOutputS = str(layerOutput)  #tf.tensor.toString(layerOutput)    #layerOutput.tostring()
        testwritefile.write(layerOutputS)
    testwritefile.close()

## Evaluate model

In [276]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 2.3011 - accuracy: 0.1135 - 605ms/epoch - 2ms/step


[2.301072835922241, 0.11349999904632568]

In [277]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [278]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.09987641, 0.10132239, 0.09984112, 0.10027289, 0.09975263,
        0.09898473, 0.09987532, 0.10049849, 0.09955052, 0.10002548],
       [0.09987641, 0.10132239, 0.09984112, 0.10027289, 0.09975263,
        0.09898473, 0.09987532, 0.10049849, 0.09955052, 0.10002548],
       [0.09987641, 0.10132239, 0.09984112, 0.10027289, 0.09975263,
        0.09898473, 0.09987532, 0.10049849, 0.09955052, 0.10002548],
       [0.09987641, 0.10132239, 0.09984112, 0.10027289, 0.09975263,
        0.09898473, 0.09987532, 0.10049849, 0.09955052, 0.10002548],
       [0.09987641, 0.10132239, 0.09984112, 0.10027289, 0.09975263,
        0.09898473, 0.09987532, 0.10049849, 0.09955052, 0.10002548]],
      dtype=float32)>